In [40]:
import pandas as pd
import numpy as np
import sklearn
import torch.nn as nn
import sklearn.model_selection
from sklearn import linear_model
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
import pickle
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)
pd.reset_option('display.max_rows')

In [29]:
def add_target(team):
    team['target'] = team['won'].shift(-1)
    return team

def rest_days(team):
    team['date_time'] = pd.to_datetime(team['date'])
    team['rest'] = (team['date_time'] - team['date_time'].shift(1)).dt.days
    team['rest'] = team['rest'].fillna(0)
    team['rest'] = team['rest'].astype(int)
    del team['date_time']
    return team

def winrate(team):
    total = team['Wins'] + team['Losses']
    total_opp = team['Wins_opp'] + team['Losses_opp']
    team['winrate'] = team['Wins'] / total
    team['winrate_opp'] = team['Wins_opp'] / total_opp
    return team

def differential(team):
    team['differential'] = team['Total'] - team['Total_opp']
    return team

def find_team_exp_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=5, adjust=False).mean()
    return rolling

def find_team_exp_average_9(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=9, adjust=False).mean()
    return rolling

def find_team_exp_average_12(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.ewm(span=12, adjust=False).mean()
    return rolling

def find_team_average_15(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(15).mean()
    return rolling

def find_team_average_10(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(10).mean()
    return rolling

def find_team_average_5(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(5).mean()
    return rolling

def find_team_average_3(team):
    numeric_columns = team.select_dtypes(include=np.number)
    rolling = numeric_columns.rolling(3).mean()
    return rolling

def rolling(data):
    df_rolling_3 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_3 = df_rolling_3.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_3)
    df_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_5 = df_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_5)
    df_rolling_10 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_10 = df_rolling_10.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_10)
    df_rolling_15 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_rolling_15 = df_rolling_15.groupby(['Teams', 'season'], group_keys = False).apply(find_team_average_15)
    df_exp_rolling_5 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_5 = df_exp_rolling_5.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_5)
    df_exp_rolling_9 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_9 = df_exp_rolling_9.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_9)
    df_exp_rolling_12 = data[list(valid_columns) + ['Teams','won', "season"]]
    df_exp_rolling_12 = df_exp_rolling_12.groupby(['Teams', 'season'], group_keys = False).apply(find_team_exp_average_12)
    exp_rolling_columns_5 = [f"{col}_exp_5" for col in df_exp_rolling_5.columns]
    exp_rolling_columns_9 = [f"{col}_exp_9" for col in df_exp_rolling_9.columns]
    exp_rolling_columns_12 = [f"{col}_exp_12" for col in df_exp_rolling_12.columns]
    rolling_columns_15 = [f"{col}_15" for col in df_rolling_15.columns]
    rolling_columns_10 = [f"{col}_10" for col in df_rolling_10.columns]
    rolling_columns_5 = [f"{col}_5" for col in df_rolling_5.columns]
    rolling_columns_3 = [f"{col}_3" for col in df_rolling_3.columns]
    df_exp_rolling_12.columns = exp_rolling_columns_12
    df_exp_rolling_9.columns = exp_rolling_columns_9
    df_exp_rolling_5.columns = exp_rolling_columns_5
    df_rolling_15.columns = rolling_columns_15
    df_rolling_10.columns = rolling_columns_10
    df_rolling_5.columns = rolling_columns_5
    df_rolling_3.columns = rolling_columns_3
    df = pd.concat([data, df_rolling_3, df_rolling_5, df_rolling_10, df_rolling_15, df_exp_rolling_5,df_exp_rolling_9, df_exp_rolling_12], axis=1)
    # df_exp_rolling_5,df_exp_rolling_9, df_exp_rolling_12
    return df

def ratio(feature):
    feature_opp = 'OPP_' + str(feature)
    free = nba[feature] / nba[feature_opp]
    return free

def ratios(nba):
    regard = []
    disregard = [col for col in nba.columns if "OPP_" in col]
    for col in disregard:
        col = col[4:100]
        if col in nba.columns:
            regard.append(col)
    nba_ratio = nba[regard].apply(ratio)
    nba_ratios_columns = [f"{col}_ratio" for col in nba_ratio.columns]
    nba_ratio.columns
    return regard

def shift_col(team, col_name):
    next_col = team[col_name].shift(-1)
    return next_col

def add_col(df, col_name):
    return df.groupby("Teams", group_keys=False).apply(lambda x: shift_col(x, col_name))

def date_change(datetime_str):
    # Parse the datetime string into a datetime object
    datetime_obj = datetime.strptime(datetime_str, '%m/%d/%Y')

    # Format the datetime object into a new string structure
    new_datetime_str = datetime_obj.strftime('%Y-%m-%d')

    return new_datetime_str

def haircut(df, date):
    df[date] = df[date].str[:10]
    return df

def convert_date_format(df):
    # Create a boolean mask to identify values in the "m/d/y" format
    mask = df['Date'].str.contains(r'\d{1,2}/\d{1,2}/\d{2}')
    
    # Apply the conversion only to values that match the mask
    df.loc[mask, 'Date'] = nba.loc[mask, 'Date'].apply(date_change)
    return df

def spread(df):
    # construct spread between home and away rankings
    ranks = [i for i in df.columns if 'RANK' in i]
    ranks_home = [i for i in ranks if '_x' in i]
    ranks_away = [i for i in ranks if '_y' in i]
    spread_columns_names = [col[:-1] for col in ranks_home]
    
    spread_columns = []
    for base_name in spread_columns_names:
        spread_column = df[f'{base_name}x'] - df[f'{base_name}y']
        spread_column.name = f'{base_name}spread'
        spread_columns.append(spread_column)
        
    spread_df = pd.concat(spread_columns, axis=1)
    return spread_df

def last_season(df):
    # construct winrate from last season
    df['last_season_winrate'] = df.groupby('Teams', 'season')['winrate'].last()

In [58]:
folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/raw_data/NBA_2018_2024.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/data/raw_data/NBA_2018_2024.csv"

df = pd.read_csv(folder_path, index_col=0)

folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/nba_api/data/teams_stats/processed_cum_2018_2024.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/nba_api/data/teams_stats/processed_cumulative_season_stats_2019_2024.csv"
nba = pd.read_csv(folder_path, index_col=0)

folder_path = "/Users/benjamincheng/Documents/GitHub/Sports-Betting/data/odds_data/2021_01_20_onward.csv"
#folder_path = "/Users/liqingyang/Documents/GitHub/sports_trading/sports_betting/data/odds_data/2021_01_20_onward.csv"
odds = pd.read_csv(folder_path, index_col=0)

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_85291/3122641595.py:8: DtypeWarning: Columns (240) have mixed types. Specify dtype option on import or set low_memory=False.
  nba = pd.read_csv(folder_path, index_col=0)


In [59]:
# nba dataframe does not include the 2018 season
df = df[~df['season'].isin([2018])]
df = df.reset_index(drop=True)
df = haircut(df, 'date')

# rename nba columns to match df
nba = haircut(nba, 'Date')
nba = convert_date_format(nba)
nba.rename(columns={'Date': 'date'}, inplace=True)

#rename odds columns to match df
odds.rename(columns={'Timestamp': 'date_next'}, inplace=True)

# construct winrate for team
df = winrate(df)
# construct differential points
df = differential(df)
# construct target
df = df.groupby("Teams", group_keys=False).apply(add_target)
# construct resting
df = df.groupby(["Teams",'season'], group_keys=False).apply(rest_days)
# games yet to play are 2
df.loc[pd.isnull(df['target']), 'target'] = 2
# convert win/loss to 1/0
df['target'] = df['target'].astype(int)

# remove metadata and target for df
removed = ['target', 'date', 'Teams_opp', 'Teams',
           'season','won', 'Wins', 'Losses', 
           'Wins_opp', 'Losses_opp']
valid_columns = df.columns[~df.columns.isin(removed)]

# scale the data for df
scaler = MinMaxScaler()
df[valid_columns] = scaler.fit_transform(df[valid_columns])

# construct rolling features to df
df = rolling(df).copy()
df = df.dropna()

# remove metadata for nba ranking 
removed = ['date', 'Teams']
valid_columns = nba.columns[~nba.columns.isin(removed)]

# scale the ranking data
scaler = MinMaxScaler()
nba[valid_columns] = scaler.fit_transform(nba[valid_columns])

# remove metadata for odds data
# removed = list(odds.columns[odds.dtypes == 'object'])
# valid_columns = odds.columns[~nba.columns.isin(removed)]

# scale the odds data
# scaler = MinMaxScaler()
# odds[valid_columns] = scaler.fit_transform(odds[valid_columns])

# construct current game metadata for df
df['home_next'] = add_col(df, 'home')
df['team_next_opp'] = add_col(df, 'Teams_opp')
df['date_next'] = add_col(df, 'date')
df = df.copy()

full = pd.merge(df, nba, on=['Teams', 'date'], how='left')

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_85291/1531626907.py:19: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby("Teams", group_keys=False).apply(add_target)
/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_85291/1531626907.py:21: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df = df.groupby(["Teams",'season'], group_keys=False).apply(rest_days)
/var/folders/99/zv3w

## Construction of Future Games
We need to now construct a subset of the dataframe to reflect future games.

In [60]:
mapping = {'Brooklyn Nets': 'BRK', 'Golden State Warriors': 'GSW', 'Los Angeles Lakers': 'LAL',
       'Milwaukee Bucks': 'MIL', 'Boston Celtics': 'BOS', 'Charlotte Hornets': 'CHO',
       'Chicago Bulls': 'CHI', 'Cleveland Cavaliers': 'CLE', 'Denver Nuggets': 'DEN',
       'Detroit Pistons': 'DET', 'Houston Rockets': 'HOU', 'Indiana Pacers': 'IND',
       'Memphis Grizzlies': 'MEM', 'Minnesota Timberwolves': 'MIN',
       'New Orleans Pelicans': 'NOP', 'New York Knicks': 'NYK', 'Oklahoma City Thunder': 'OKC',
       'Orlando Magic': 'ORL', 'Philadelphia 76ers': 'PHI', 'Phoenix Suns': 'PHO',
       'Portland Trail Blazers': 'POR', 'Sacramento Kings': 'SAC', 'San Antonio Spurs': 'SAS',
       'Toronto Raptors': 'TOR', 'Utah Jazz': 'UTA', 'Washington Wizards': 'WAS',
       'Atlanta Hawks': 'ATL', 'Dallas Mavericks': 'DAL', 'LA Clippers': 'LAC', 'Miami Heat': 'MIA'}

In [1]:
# Need to update this 

future_teams = ['LAL', 'WAS', 'POR', 'CHO', 'IND', 'BRK', 'DET', 'ATL', 'OKC', 'BOS', 'MEM', 'MIL', 'ORL', 'NOP', 'TOR', 'MIN', 'CLE', 'PHO']

In [2]:
def construction(df, teams):
    constructed = []
    # teams are the teams that will play in the future
    for team in teams:
        constructed.append(df[df['Teams'] == team].iloc[-1])
    tonight = pd.concat(constructed, axis=1).T
    tonight = tonight.reset_index(drop=True)
    return tonight

def opponent(tonight):
    figure = []
    for i in range(len(tonight['Teams'])):
        if (i % 2 == 0):
            figure.append(tonight['Teams'][i+1])
        else:
            figure.append(tonight['Teams'][i-1])
    return figure

In [3]:
tonight = construction(full, future_teams)

NameError: name 'full' is not defined

In [64]:
tonight['home_next'] = [0,1] * 9 # need to update this
tonight['team_next_opp'] = opponent(tonight)
date_to_repeat = '2024-04-03' # need to update this
tonight['date_next'] = [date_to_repeat] * len(tonight)

In [65]:
# merge stats from opposing teams
complete = tonight.merge(tonight,
               left_on=['Teams', 'date_next'],
               right_on = ['team_next_opp', 'date_next'])

# concat the spreads
spread_df = spread(complete)
complete = pd.concat([complete, spread_df], axis=1)

# concat the odds
complete = pd.merge(complete, odds, on=['Teams_x', 'date_next', 'Teams_y'], how='left')
complete = complete.dropna()

In [66]:
complete

mp_x      fg_x     fga_x     fg%_x      3p_x     3pa_x     3p%_x      ft_x  \
0   0.0       0.5      0.25  0.674641  0.222222       0.3  0.280724  0.619048   
1   0.0  0.431818  0.416667  0.444976  0.333333  0.466667  0.284605  0.333333   
2   0.0  0.477273  0.233333  0.660287  0.481481       0.5   0.39586  0.357143   
3   0.0  0.590909      0.45  0.605263  0.333333  0.333333  0.385511  0.309524   
4   0.0  0.386364  0.483333  0.344498  0.407407      0.55  0.301423       0.5   
5   0.0  0.431818       0.3  0.538278  0.444444  0.433333  0.413972  0.547619   
6   0.0  0.522727  0.316667  0.641148  0.407407       0.4  0.404916  0.238095   
7   0.0  0.636364  0.333333  0.767943  0.814815      0.55    0.6326  0.261905   
8   0.0  0.409091  0.483333  0.370813  0.407407  0.466667  0.353169  0.333333   
9   0.0  0.386364       0.3  0.478469  0.333333      0.45  0.294955  0.285714   
10  0.0  0.431818  0.433333  0.430622  0.555556      0.55  0.421734  0.333333   
11  0.0  0.659091       0.4  0.732057  0.703704  0.383333  0.733506  0.166667   
12  0.0  0.386364  0.316667  0.464115  0.222222      0.25  0.324709  0.357143   
13  0.0  0.295455  0.333333  0.337321  0.518519  0.516667  0.415265   0.47619   
14  0.0  0.613636       0.5  0.590909  0.814815  0.616667  0.571798  0.071429   
15  0.0  0.545455  0.483333  0.526316  0.518519  0.483333  0.441138  0.214286   

       fta_x     ft%_x     orb_x     drb_x     trb_x     ast_x     stl_x  \
0   0.627451   0.72863  0.206897  0.409091  0.346154  0.452381  0.318182   
1   0.352941  0.677069  0.241379  0.431818  0.384615  0.452381  0.272727   
2   0.372549  0.691995  0.206897  0.636364  0.538462  0.380952  0.363636   
3   0.333333  0.660787  0.241379  0.159091  0.153846  0.547619  0.454545   
4   0.607843   0.56038  0.517241  0.340909  0.461538  0.380952  0.363636   
5   0.509804  0.812754  0.206897  0.454545  0.384615  0.333333       0.5   
6   0.411765  0.321574  0.310345  0.409091  0.403846       0.5  0.272727   
7   0.254902  0.746269  0.310345  0.318182  0.326923   0.47619  0.272727   
8   0.313725  0.785617  0.413793  0.340909  0.403846  0.333333  0.409091   
9   0.352941  0.548168  0.275862  0.295455  0.288462   0.47619  0.545455   
10  0.294118  0.849389  0.551724  0.295455  0.442308  0.547619  0.318182   
11  0.196078   0.58209   0.37931  0.545455  0.557692  0.666667  0.318182   
12  0.352941  0.742198  0.206897  0.136364  0.115385  0.428571  0.454545   
13  0.490196  0.709634   0.37931  0.136364  0.211538  0.333333  0.272727   
14  0.117647  0.397558  0.344828  0.477273  0.480769  0.380952  0.318182   
15  0.196078  0.791045  0.413793  0.295455  0.365385  0.619048  0.681818   

   blk_x     tov_x     pf_x     pts_x     ts%_x    efg%_x    3par_x     ftr_x  \
0   0.25  0.310345    0.375       0.5   0.65708  0.611336  0.428816  0.589235   
1   0.05  0.482759      0.5  0.361111  0.429204  0.437247  0.554031  0.291785   
2    0.3   0.62069  0.28125  0.444444  0.692478  0.692308  0.703259  0.358357   
3    0.1   0.37931     0.25  0.481481   0.55531   0.57085   0.38422  0.267705   
4   0.15  0.310345  0.40625  0.407407  0.371681  0.370445  0.614065  0.477337   
5    0.4  0.413793  0.40625  0.472222  0.617257  0.564777  0.574614   0.46034   
6    0.4   0.37931  0.53125  0.416667  0.561947  0.637652  0.524871  0.368272   
7    0.5  0.482759  0.34375   0.62037  0.871681  0.878543  0.701544  0.226629   
8   0.15  0.310345     0.25  0.361111  0.391593  0.392713  0.521441  0.246459   
9   0.25  0.413793    0.125  0.305556  0.451327  0.477733  0.596913  0.320113   
10  0.25   0.62069   0.1875  0.416667  0.502212  0.493927  0.641509  0.239377   
11   0.2  0.517241  0.40625  0.574074  0.763274  0.797571  0.463122  0.164306   
12   0.2  0.482759   0.1875  0.305556  0.435841  0.427126  0.336192  0.315864   
13   0.2   0.37931    0.375  0.351852  0.442478   0.41498  0.660377  0.430595   
14   0.4  0.413793  0.34375  0.527778  0.639381  0.696356  0.679245  0.090652   
15  0.15  0.482759     0.25  0.45370

In [67]:
# load models and features
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/factors/'
file_name = 'calibrated_predictors_ridge_classifier_30_predictors_19.97%_2019_2024.txt'
file_path = save_path + file_name
with open(file_path, 'r') as f:
    predictors = f.read()
    
# model
save_path = '/Users/benjamincheng/Documents/GitHub/Sports-Betting/ml_notebooks/weights/'
file_name = 'calibrated_ridge_classifier_30_predictors_19.97%_2019_2024.pkl'
file_path = save_path + file_name
with open(file_path, 'rb') as f:
    clf = pickle.load(f)

predictors = predictors.split(',')[:-1]
X = complete[predictors]
predictions = clf.predict(X)
result = pd.Series(predictions, index = X.index)
final = pd.concat([complete['Teams_x'], result], axis=1)

In [68]:
probabilities = clf.predict_proba(X)[:, 1]
result = pd.DataFrame({
    'Predictions': predictions,
    'Probabilities': probabilities,
    'Fanduel': complete['Fanduel_odds_x'],
    'Draftkings': complete['Draftkings_odds_x'],
    'BetMGM': complete['BetMGM_odds_x']
}, index=X.index)

# Concatenate the predictions and probabilities with the original data
final = pd.concat([complete['date_next'],complete['Teams_x'], result], axis=1)
final.columns = ['Date','Teams', 'Predictions', 'Probabilities', 'Fanduel', 'Draftkings', 'BetMGM']
final['Our_Odds'] = 1 / final['Probabilities']
final['Best_Odds'] = final[['Fanduel', 'Draftkings', 'BetMGM']].max(axis=1)

In [86]:
from sklearn.metrics import accuracy_score

final['Actual'] = [0,1,1,0,0,1,0,1,0,1,0,1,1,0,0,1,]
accuracy = accuracy_score(final['Actual'], final['Predictions'])
accuracy

0.6875

In [87]:
final

,Date,Teams,Predictions,Probabilities,Fanduel,Draftkings,BetMGM,Our_Odds,Best_Odds,Actual
0,2024-04-02,MIL,1,1.000000,1.11,1.11,1.10,1.000000,1.11,0
1,2024-04-02,WAS,0,0.180000,7.40,7.00,7.25,5.555556,7.40,1
2,2024-04-02,LAL,1,0.785714,1.11,1.12,1.11,1.272727,1.12,1
3,2024-04-02,TOR,0,0.180000,7.10,6.75,6.75,5.555556,7.10,0
4,2024-04-02,NYK,0,0.395455,2.24,2.24,2.25,2.528736,2.25,0
5,2024-04-02,MIA,1,0.581818,1.68,1.68,1.67,1.718750,1.68,1
6,2024-04-02,OKC,1,0.620690,1.45,1.49,1.48,1.611111,1.49,0
7,2024-04-02,PHI,0,0.395455,2.80,2.70,2.70,2.528736,2.80,1
8,2024-04-02,HOU,0,0.264901,3.45,3.30,3.30,3.775000,3.45,0
9,2024-04-02,MIN,1,0.727273,1.33,1.35,1.33,1.375000,1.35,1


In [70]:
win = final[final['Predictions'] == 1]
bet = win[win['Our_Odds'] < win['Best_Odds']]
bet['odds_diff'] = bet['Best_Odds'] - bet['Our_Odds']

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_85291/3053637661.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bet['odds_diff'] = bet['Best_Odds'] - bet['Our_Odds']


In [73]:
def kelly(decimal_odds, P, fractional):
    # shrinkage is the percent we want to shrink bets to account for uncertanity
    B = decimal_odds - 1
    Q = 1 - P
    solution = (B*P - Q) / B
    solution = solution * fractional
    return solution

In [74]:
bet['Kelly'] = kelly(bet['Best_Odds'], bet['Probabilities'], 0.5)

/var/folders/99/zv3w4flx1598dxlknc099kfr0000gn/T/ipykernel_85291/2365977069.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  bet['Kelly'] = kelly(bet['Best_Odds'], bet['Probabilities'], 0.5)


In [78]:
bet_very_undervalued = bet[bet['odds_diff'] > 0.06]
bet_very_undervalued = bet_very_undervalued[bet_very_undervalued['Best_Odds'] > 1.1]

In [79]:
bet_very_undervalued

,Date,Teams,Predictions,Probabilities,Fanduel,Draftkings,BetMGM,Our_Odds,Best_Odds,odds_diff,Kelly
0,2024-04-02,MIL,1,1.000000,1.11,1.11,1.10,1.00000,1.11,0.11000,0.500000
14,2024-04-02,DAL,1,0.543478,1.96,1.91,1.95,1.84000,1.96,0.12000,0.033967
15,2024-04-02,GSW,1,0.581818,1.89,1.91,1.87,1.71875,1.91,0.19125,0.061139
